In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost
from xgboost import plot_importance

In [2]:
train = pd.read_csv('Train_maskedv2.csv')
train.head()

,ward,total_households,total_individuals,target_pct_vunerable,dw_00,dw_01,dw_02,dw_03,dw_04,dw_05,...,lan_11,lan_12,lan_13,lan_14,pg_00,pg_01,pg_02,pg_03,pg_04,lgt_00
0,9D9BEUB,13569.97801,39024.03083,1.739801,0.354806,0.004781,0.170498,0.007861,0.006796,0.023082,...,0.001723,0.027796,0,0.026727,0.524666,0.041460,0.394550,0.029031,0.010292,0.599259
1,RERH3XM,13593.88256,32879.94646,1.993381,0.262140,0.002066,0.077669,0.000597,0.000977,0.003759,...,0.041982,0.021312,0,0.015754,0.993900,0.001829,0.000854,0.000586,0.002832,0.699136
2,GJWA3BO,2698.30050,8261.71093,0.038340,0.728101,0.002292,0.051663,0.008209,0.001444,0.005089,...,0.014786,0.005827,0,0.049384,0.990007,0.003582,0.002546,0.003201,0.000663,0.972315
3,GW6VRB7,1965.96698,9069.77694,7.411006,0.613206,0.138127,0.004999,0.002704,0.000000,0.002788,...,0.003087,0.003136,0,0.013562,0.928682,0.004464,0.009854,0.054701,0.002299,0.735333
4,CN9JZJC,1852.07821,9908.84404,8.223907,0.797462,0.120830,0.060677,0.001155,0.001042,0.000508,...,0.000117,0.001471,0,0.000000,0.994072,0.000476,0.004033,0.000704,0.000714,0.646964


In [3]:
test=pd.read_csv('Test_maskedv2.csv')
test.head()

,ward,total_households,total_individuals,dw_00,dw_01,dw_02,dw_03,dw_04,dw_05,dw_06,...,lan_11,lan_12,lan_13,lan_14,pg_00,pg_01,pg_02,pg_03,pg_04,lgt_00
0,00A93O0,2514.04172,8037.06199,0.671962,0.061074,0.101516,0.006891,0.021287,0.015615,0.035974,...,0.000420,0.011485,0,0.063705,0.979272,0.009926,0.003224,0.002697,0.004882,0.521007
1,01V40KO,1829.71944,7494.86917,0.542857,0.390850,0.039623,0.000000,0.000000,0.000000,0.002633,...,0.000000,0.000160,0,0.000000,0.998938,0.000296,0.000301,0.000305,0.000160,0.693716
2,01ZHVW0,1721.99009,5847.43254,0.968071,0.003525,0.001431,0.000000,0.000000,0.000000,0.018546,...,0.000631,0.006504,0,0.000171,0.955895,0.012170,0.000000,0.030012,0.001923,0.933954
3,028YQJX,12388.02257,52211.66859,0.397274,0.004734,0.001068,0.000279,0.000799,0.197760,0.005227,...,0.000593,0.008330,0,0.000000,0.293629,0.694779,0.003238,0.000628,0.007726,0.921470
4,02G8W11,1754.25716,4266.57609,0.815197,0.016697,0.078571,0.008975,0.047882,0.015398,0.007264,...,0.000234,0.016846,0,0.178598,0.110861,0.054114,0.006062,0.823495,0.005470,0.996227


In [4]:



#using a heatmap to study numeric fearturs
train_data = train.drop(['ward'],axis=1)
test_data = test.drop(['ward'],axis=1)

In [5]:
train_data['total_households']=train_data['total_households']*1000
train_data['total_households']=train_data['total_households'].apply(np.floor)
train_data['total_individuals']=train_data['total_individuals']*1000
train_data['total_individuals']=train_data['total_individuals'].apply(np.floor)
test_data['total_households']=test_data['total_households']*1000
test_data['total_households'].apply(np.floor)
test_data['total_individuals']=test_data['total_individuals']*1000
test_data['total_individuals'].apply(np.floor)
train_data.head()

,total_households,total_individuals,target_pct_vunerable,dw_00,dw_01,dw_02,dw_03,dw_04,dw_05,dw_06,...,lan_11,lan_12,lan_13,lan_14,pg_00,pg_01,pg_02,pg_03,pg_04,lgt_00
0,13569978.0,39024030.0,1.739801,0.354806,0.004781,0.170498,0.007861,0.006796,0.023082,0.014743,...,0.001723,0.027796,0,0.026727,0.524666,0.041460,0.394550,0.029031,0.010292,0.599259
1,13593882.0,32879946.0,1.993381,0.262140,0.002066,0.077669,0.000597,0.000977,0.003759,0.099883,...,0.041982,0.021312,0,0.015754,0.993900,0.001829,0.000854,0.000586,0.002832,0.699136
2,2698300.0,8261710.0,0.038340,0.728101,0.002292,0.051663,0.008209,0.001444,0.005089,0.059432,...,0.014786,0.005827,0,0.049384,0.990007,0.003582,0.002546,0.003201,0.000663,0.972315
3,1965966.0,9069776.0,7.411006,0.613206,0.138127,0.004999,0.002704,0.000000,0.002788,0.007380,...,0.003087,0.003136,0,0.013562,0.928682,0.004464,0.009854,0.054701,0.002299,0.735333
4,1852078.0,9908844.0,8.223907,0.797462,0.120830,0.060677,0.001155,0.001042,0.000508,0.002171,...,0.000117,0.001471,0,0.000000,0.994072,0.000476,0.004033,0.000704,0.000714,0.646964


In [6]:
#convert select columns to intergers
train_data['total_households']=train_data['total_households'].astype(np.int64)
train_data['total_individuals']=train_data['total_individuals'].astype(np.int64)
test_data['total_households']=test_data['total_households'].astype(np.int64)
test_data['total_individuals']=test_data['total_individuals'].astype(np.int64)

In [7]:
train_data.head()

,total_households,total_individuals,target_pct_vunerable,dw_00,dw_01,dw_02,dw_03,dw_04,dw_05,dw_06,...,lan_11,lan_12,lan_13,lan_14,pg_00,pg_01,pg_02,pg_03,pg_04,lgt_00
0,13569978,39024030,1.739801,0.354806,0.004781,0.170498,0.007861,0.006796,0.023082,0.014743,...,0.001723,0.027796,0,0.026727,0.524666,0.041460,0.394550,0.029031,0.010292,0.599259
1,13593882,32879946,1.993381,0.262140,0.002066,0.077669,0.000597,0.000977,0.003759,0.099883,...,0.041982,0.021312,0,0.015754,0.993900,0.001829,0.000854,0.000586,0.002832,0.699136
2,2698300,8261710,0.038340,0.728101,0.002292,0.051663,0.008209,0.001444,0.005089,0.059432,...,0.014786,0.005827,0,0.049384,0.990007,0.003582,0.002546,0.003201,0.000663,0.972315
3,1965966,9069776,7.411006,0.613206,0.138127,0.004999,0.002704,0.000000,0.002788,0.007380,...,0.003087,0.003136,0,0.013562,0.928682,0.004464,0.009854,0.054701,0.002299,0.735333
4,1852078,9908844,8.223907,0.797462,0.120830,0.060677,0.001155,0.001042,0.000508,0.002171,...,0.000117,0.001471,0,0.000000,0.994072,0.000476,0.004033,0.000704,0.000714,0.646964


In [8]:
test_data.head()

,total_households,total_individuals,dw_00,dw_01,dw_02,dw_03,dw_04,dw_05,dw_06,dw_07,...,lan_11,lan_12,lan_13,lan_14,pg_00,pg_01,pg_02,pg_03,pg_04,lgt_00
0,2514041,8037061,0.671962,0.061074,0.101516,0.006891,0.021287,0.015615,0.035974,0.010995,...,0.000420,0.011485,0,0.063705,0.979272,0.009926,0.003224,0.002697,0.004882,0.521007
1,1829719,7494869,0.542857,0.390850,0.039623,0.000000,0.000000,0.000000,0.002633,0.005074,...,0.000000,0.000160,0,0.000000,0.998938,0.000296,0.000301,0.000305,0.000160,0.693716
2,1721990,5847432,0.968071,0.003525,0.001431,0.000000,0.000000,0.000000,0.018546,0.004769,...,0.000631,0.006504,0,0.000171,0.955895,0.012170,0.000000,0.030012,0.001923,0.933954
3,12388022,52211668,0.397274,0.004734,0.001068,0.000279,0.000799,0.197760,0.005227,0.065487,...,0.000593,0.008330,0,0.000000,0.293629,0.694779,0.003238,0.000628,0.007726,0.921470
4,1754257,4266576,0.815197,0.016697,0.078571,0.008975,0.047882,0.015398,0.007264,0.000000,...,0.000234,0.016846,0,0.178598,0.110861,0.054114,0.006062,0.823495,0.005470,0.996227


In [9]:
train_data.describe()

,total_households,total_individuals,target_pct_vunerable,dw_00,dw_01,dw_02,dw_03,dw_04,dw_05,dw_06,...,lan_11,lan_12,lan_13,lan_14,pg_00,pg_01,pg_02,pg_03,pg_04,lgt_00
count,3.174000e+03,3.174000e+03,3174.000000,3174.000000,3174.000000,3174.000000,3174.000000,3174.000000,3174.000000,3174.000000,...,3174.000000,3174.000000,3174.0,3174.000000,3174.000000,3174.000000,3174.000000,3174.000000,3174.000000,3174.000000
mean,3.429319e+06,1.240768e+07,10.258947,0.710593,0.097372,0.031894,0.005820,0.008202,0.006047,0.022225,...,0.050327,0.012342,0.0,0.014335,0.863095,0.041924,0.017665,0.073692,0.003624,0.830599
std,3.117343e+06,9.310300e+06,12.847192,0.217458,0.188484,0.078634,0.018592,0.029408,0.023772,0.036762,...,0.153493,0.020594,0.0,0.041155,0.248096,0.146524,0.074607,0.169049,0.010378,0.208129
min,1.000000e+03,4.020000e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.011053,0.000000,0.000000,0.000000,0.000000,0.001692
25%,1.668367e+06,6.972024e+06,0.429761,0.591480,0.002919,0.002421,0.000000,0.000000,0.000000,0.002676,...,0.000453,0.002477,0.0,0.000000,0.879603,0.000960,0.000848,0.000445,0.000633,0.788085
50%,2.263921e+06,9.217504e+06,3.715314,0.766925,0.010649,0.005792,0.000803,0.000603,0.000813,0.008408,...,0.003092,0.006266,0.0,0.000097,0.989811,0.003088,0.001753,0.001206,0.001630,0.909446
75%,3.681155e+06,1.346354e+07,17.762604,0.885571,0.074650,0.028085,0.002538,0.002145,0.002921,0.025111,...,0.028330,0.013430,0.0,0.011548,0.995798,0.012419,0.004636,0.039255,0.004025,0.963780
max,3.959520e+07,9.171675e+07,100.000000,0.994962,0.931490,0.951806,0.264239,0.392085,0.435912,0.412936,...,0.991674,0.367785,0.0,0.998448,1.000000,0.969519,0.939564,0.940563,0.367842,1.000000


In [10]:
test_data.describe()

,total_households,total_individuals,dw_00,dw_01,dw_02,dw_03,dw_04,dw_05,dw_06,dw_07,...,lan_11,lan_12,lan_13,lan_14,pg_00,pg_01,pg_02,pg_03,pg_04,lgt_00
count,1.102000e+03,1.102000e+03,1102.000000,1102.000000,1102.000000,1102.000000,1102.000000,1102.000000,1102.000000,1102.000000,...,1102.000000,1102.000000,1102.0,1102.000000,1102.000000,1102.000000,1102.000000,1102.000000,1102.000000,1102.000000
mean,2.954164e+06,1.123846e+07,0.554544,0.237620,0.045786,0.003852,0.005205,0.026109,0.026799,0.030418,...,0.000697,0.008765,0.0,0.022219,0.672871,0.228805,0.004585,0.087391,0.006347,0.791468
std,2.668062e+06,8.912096e+06,0.282261,0.314265,0.078767,0.009472,0.015252,0.067398,0.040966,0.049703,...,0.001423,0.014138,0.0,0.060707,0.395152,0.324664,0.013210,0.184184,0.010378,0.258186
min,1.060830e+05,3.598810e+05,0.005075,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.011043,0.000000,0.000000,0.000000,0.000000,0.001048
25%,1.598110e+06,6.445408e+06,0.309676,0.003207,0.006650,0.000000,0.000000,0.000434,0.005224,0.002588,...,0.000000,0.001222,0.0,0.000000,0.202813,0.001221,0.000713,0.000548,0.000717,0.750859
50%,1.964204e+06,8.083349e+06,0.611551,0.015616,0.019158,0.000938,0.000726,0.002011,0.012352,0.009622,...,0.000245,0.004117,0.0,0.000934,0.970641,0.012663,0.001593,0.001889,0.003037,0.888062
75%,2.903222e+06,1.152377e+07,0.801064,0.506504,0.049493,0.002594,0.002615,0.015702,0.028745,0.035531,...,0.000734,0.009616,0.0,0.018276,0.995667,0.483053,0.004150,0.075370,0.007862,0.962184
max,2.021433e+07,6.451152e+07,0.982412,0.985465,0.760766,0.110322,0.222271,0.742617,0.418048,0.440729,...,0.014121,0.125735,0.0,0.925600,0.999697,0.982037,0.281499,0.925508,0.185216,1.000000


In [11]:
model = xgboost.XGBRegressor(colsample_bytree=0.4,
                 gamma=0,                 
                 learning_rate=0.07,
                 max_depth=3,
                 min_child_weight=1.5,
                 n_estimators=10000,                                                                    
                 reg_alpha=0.75,
                 reg_lambda=0.45,
                 subsample=0.6,
                 seed=42)

In [14]:
X=train_data.drop(['target_pct_vunerable'],axis=1)
Y =train_data['target_pct_vunerable']
from sklearn.decomposition import PCA

pca = PCA()

principalComponents = pca.fit_transform(X)

X = pd.DataFrame(data = principalComponents
             , columns = X.columns)
model.fit(X,Y)

#data=pd.DataFrame(model.feature_importances_,index=X.columns)

XGBRegressor(base_score=0.5, booster=None, colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.4, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints=None,
             learning_rate=0.07, max_delta_step=0, max_depth=3,
             min_child_weight=1.5, missing=nan, monotone_constraints=None,
             n_estimators=10000, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=42, reg_alpha=0.75,
             reg_lambda=0.45, scale_pos_weight=1, seed=42, subsample=0.6,
             tree_method=None, validate_parameters=False, verbosity=None)

In [18]:
X.head()

,total_households,total_individuals,dw_00,dw_01,dw_02,dw_03,dw_04,dw_05,dw_06,dw_07,...,lan_11,lan_12,lan_13,lan_14,pg_00,pg_01,pg_02,pg_03,pg_04,lgt_00
0,2.844390e+07,1.485752e+06,0.257998,0.368814,0.059395,0.198100,0.063401,-0.099520,-0.019196,0.042051,...,5.018125e-10,3.910932e-10,3.939339e-10,3.253111e-09,3.547781e-09,-3.593467e-09,-5.134016e-10,1.030317e-09,4.541168e-10,-3.871848e-10
1,2.260346e+07,3.393501e+06,1.308506,0.190624,-0.008249,0.055963,0.160526,0.212009,-0.062474,0.045468,...,4.062076e-10,1.838357e-10,1.281842e-10,4.157253e-11,2.211981e-09,-1.075393e-09,9.090730e-11,-1.681545e-10,-2.903522e-10,-7.231613e-10
2,-4.170303e+06,5.762142e+05,0.180042,-0.243268,0.082736,-0.395313,0.324899,-0.077517,-0.085868,-0.024704,...,-2.329749e-10,-2.633759e-10,6.610656e-11,9.484327e-11,8.961648e-10,-1.136156e-10,-5.918981e-10,-8.566830e-10,-7.189693e-11,4.568406e-10
3,-3.625885e+06,-3.687163e+05,0.288073,0.387512,-0.042278,-0.091299,-0.009669,0.082116,-0.086865,-0.014802,...,5.672612e-11,-2.087915e-10,2.511352e-10,9.342438e-10,-2.087053e-09,-1.280080e-09,4.100954e-10,4.884525e-10,-3.786732e-10,-5.473260e-12
4,-2.862223e+06,-7.345372e+05,0.423217,0.364960,-0.100099,-0.100468,-0.084820,0.232412,-0.066320,-0.016485,...,-2.318877e-10,-3.564884e-11,-2.410799e-10,5.227976e-09,2.489413e-09,4.999557e-10,-7.350297e-10,-9.266181e-11,-4.356183e-10,-3.951155e-10


In [20]:
principalComponents = pca.fit_transform(test_data)

test_data= pd.DataFrame(data = principalComponents
             , columns = X.columns)

In [21]:
test_data['Prediction'] = model.predict(test_data)
filename = 'submission12_pca.csv'
pd.DataFrame({'ward': test.ward, 'target_pct_vunerable': test_data.Prediction}).to_csv(filename, index=False)

feat=[]
val =[]
for x in X.columns:
    feat.append(x)
for x in data[0]:
    val.append(x)

print(feat)
print(val)
data_dict=dict(zip(feat,val))

data_dict

X_features=[]
for i in data_dict.keys():
    if data_dict[i] > 0.0:
        X_features.append(i)
        print(i,data_dict[i])
print(X_features)

#analyse the most important features
plt.scatter(X.dw_01, Y, c = "blue", marker = "s")
plt.title("dwelling type 1 vs vulnerability")
plt.xlabel("dwelling 01")
plt.ylabel("vulnerability")
plt.show()

plt.scatter(X.psa_01, Y, c = "blue", marker = "s")
plt.title("psa 1 vs vulnerability")
plt.xlabel("psa 01")
plt.ylabel("vulnerability")
plt.show()

#dealing with outliers
train_data=train_data[train_data.psa_01<0.8]
train_data=train_data[train_data['target_pct_vunerable']<60]
train_data.head()

In [ ]:
from sklearn.preprocessing import StandardScaler
train_x=train_data[X_features]
train_x = StandardScaler().fit_transform(train_x)
train_x.shape


In [ ]:
train_y=train_data['target_pct_vunerable']

In [ ]:
best_xgb_model = xgboost.XGBRegressor(colsample_bytree=0.4,
                 gamma=0,                 
                 learning_rate=0.03,
                 max_depth=3,
                 min_child_weight=1.9,
                 n_estimators=10000,                                                                    
                 reg_alpha=0.75,
                 reg_lambda=0.45,
                 subsample=0.5,booster='dart',
                 seed=42)
best_xgb_model.fit(train_x,train_y)

In [ ]:
test_data=test_data[X_features]
# Standardizing the features
test_data = StandardScaler().fit_transform(test_data)

In [ ]:
test_data['Prediction'] = best_xgb_model.predict(test_data)
filename = 'submission12_scaled.csv'
pd.DataFrame({'ward': test.ward, 'target_pct_vunerable': test_data.Prediction}).to_csv(filename, index=False)

In [22]:
print(test_data['Prediction'].head())
print(test_data['Prediction'].count())

0     9.885611
1    13.127810
2    11.104889
3     6.969592
4     1.472939
Name: Prediction, dtype: float32
1102


import lightgbm as lgbm
model2 = lgbm.LGBMRegressor(boosting_type='gbdt', objective='regression', num_leaves=1200,
                            learning_rate=0.17,bagging_fraction=0.8, feature_fraction=0.8, reg_lambda=0.9)

model2.fit(train_x,train_y)
test_data['Prediction'] = model2.predict(test_data)
filename = 'submission7_lgbm.csv'
pd.DataFrame({'ward': test.ward, 'target_pct_vunerable': test_data.Prediction}).to_csv(filename, index=False)
